In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Dropout
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model


In [ ]:
base_model=MobileNetV2(weights='imagenet',include_top=False, input_shape=(224,224,3))

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x)
x=Dropout(0.3)(x)
x=Dense(1024,activation='relu')(x)
x=Dropout(0.3)(x)
x=Dense(512,activation='relu')(x)
x=Dropout(0.5)(x)
preds=Dense(51,activation='softmax')(x)

In [ ]:

model=Model(inputs=base_model.input,outputs=preds)


In [ ]:

for i,layer in enumerate(model.layers):
  print(i,layer.name)


In [ ]:
for layer in model.layers[:156]:
    layer.trainable=False
for layer in model.layers[156:]:
    layer.trainable=True


In [ ]:
batch_size = 32
IMG_SIZE = 224
TRAINDIR = "/home/rekca/Datasets/Caltech/Zavrsni2/Train"
TESTDIR = "/home/rekca/Datasets/Caltech/Zavrsni2/Test"

train_datagen = ImageDataGenerator( 
    preprocessing_function=preprocess_input,
    horizontal_flip=True,
    rotation_range=40, 
    width_shift_range=0.1,
    height_shift_range=0.1, 
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode ='nearest')

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
        TRAINDIR,  
        target_size=(IMG_SIZE, IMG_SIZE),  
        color_mode='rgb',
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True) 

validation_generator = test_datagen.flow_from_directory(
        TESTDIR,
        target_size=(IMG_SIZE,IMG_SIZE),
        color_mode='rgb',
        batch_size=batch_size,
        class_mode='categorical',
        shuffle = True)

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
filepath = "mobile2_4.{epoch:02d}-acc_{val_acc:.2f}_loss_{val_loss:.2f}.hdf5" 

checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', save_best_only=True,
                            mode='min', period=1)

callBacks = [checkpoint]

In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=5000 // batch_size,
        epochs=30,
        validation_data=validation_generator,
        validation_steps=500 // batch_size, callbacks = callBacks)


In [ ]:
model.save("MobileNetV2.model")

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model_file('mobile2_4.19-acc_0.86_loss_0.55.hdf5')
tflite_model = converter.convert()
open("Mobile2_4.tflite", "wb").write(tflite_model)


In [ ]:
label_map = (validation_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v

print(label_map)